<a href="https://colab.research.google.com/github/Oumar199/NLP_turi_and_co/blob/main/github_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start

Text Analytics Tutorial using Sherlock Holmes Stories
----------------

#### Sections
- [Setup](####Setup)
- [Preparing and validating the dataset]()
- [Calculating Various Statistics]()
- [Constructing Social Networks]()
- [Topic Models]()

#### Setup

In [5]:
# install requirements
!pip install --upgrade gensim
!pip install --upgrade nltk
!pip install --upgrade turicreate
!pip install --upgrade pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached llvmlite-0.33.0-cp38-cp38-manylinux1_x86_64.whl (18.3 MB)
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.1
    Uninstalling llvmlite-0.39.1:
      Successfully uninstalled llvmlite-0.39.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.8.1 requires resampy>=0.2.2, but you have resampy 0.2.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Preparing the Dataset

##### Constructing the Dataset

In [6]:
# BASE_DIR = "/Users/pablo/fromHomeMax/sherlock/data"

import re
import urllib
import turicreate as tr

books_url = "https://sherlock-holm.es/ascii/"
re_books_links = re.compile("\"piwik_download\"\s+href=\"(?P<link>.*?)\">(?P<title>.*?)</a>", re.MULTILINE)
html = urllib.request.urlopen(books_url).read().decode('utf-8')
books_list = [m.groupdict() for m in re_books_links.finditer(html)]
print(books_list)

[{'link': '/stories/plain-text/cano.txt', 'title': 'The Complete Canon'}, {'link': '/stories/plain-text/advs.txt', 'title': 'The Adventures of Sherlock Holmes'}, {'link': '/stories/plain-text/mems.txt', 'title': 'The Memoirs of Sherlock Holmes'}, {'link': '/stories/plain-text/retn.txt', 'title': 'The Return of Sherlock Holmes'}, {'link': '/stories/plain-text/lstb.txt', 'title': 'His Last Bow'}, {'link': '/stories/plain-text/case.txt', 'title': 'The Case-Book of Sherlock Holmes'}, {'link': '/stories/plain-text/stud.txt', 'title': 'A Study In Scarlet'}, {'link': '/stories/plain-text/sign.txt', 'title': 'The Sign of the Four'}, {'link': '/stories/plain-text/houn.txt', 'title': 'The Hound of the Baskervilles'}, {'link': '/stories/plain-text/vall.txt', 'title': 'The Valley of Fear'}, {'link': '/stories/plain-text/scan.txt', 'title': 'A Scandal in Bohemia'}, {'link': '/stories/plain-text/redh.txt', 'title': 'The Red-Headed League'}, {'link': '/stories/plain-text/iden.txt', 'title': 'A Case o

In [7]:
# let's download the books' texts.
filtered_books = set(["The Complete Canon", "The Case-Book of Sherlock Holmes", "The Canon - U.S. edition"]) # remove some books
books_list = [element for element in books_list if element['title'] not in filtered_books]

for d in books_list:
    d['text'] = urllib.request.urlopen("http://sherlock-holm.es" + d['link']).read().strip() # recuperate contents inside the links



#### Calculating Various Statistics

Save the data to use it with turicreate.

In [8]:
data_dir = "/content/drive/MyDrive/Intelligence artificielle /Partie III Master/Traitement automatique des langues /Github NLP turicreate/data"
sf = tr.SFrame(books_list).unpack("X1", column_name_prefix = "")
sf.save("%s/books.sframe" % data_dir)

In [9]:
# Let's calculate the length of each text
sf['chars_num'] = sf['text'].apply(lambda t: len(t))
sf.head(10)

link,text,title,chars_num
/stories/plain-text/advs.txt ...,THE ADVENTURES OFSHERLOCK HOLMES\n\n ...,The Adventures ofSherlock Holmes ...,610836
/stories/plain-text/mems.txt ...,THE MEMOIRS OF SHERLOCKHOLMES\n\n ...,The Memoirs of SherlockHolmes ...,511733
/stories/plain-text/retn.txt ...,THE RETURN OF SHERLOCKHOLMES\n\n ...,The Return of SherlockHolmes ...,662236
/stories/plain-text/lstb.txt ...,HIS LAST BOW\n\nArthur Conan ...,His Last Bow,402289
/stories/plain-text/stud.txt ...,A STUDY IN SCARLET\n\nArthur Conan ...,A Study In Scarlet,260045
/stories/plain-text/sign.txt ...,THE SIGN OF THE FOUR\n\nArthur Conan ...,The Sign of the Four,251806
/stories/plain-text/houn.txt ...,THE HOUND OF THEBASKERVILLES\n\n ...,The Hound of theBaskervilles ...,346398
/stories/plain-text/vall.txt ...,THE VALLEY OF FEAR\n\nArthur Conan ...,The Valley of Fear,340222
/stories/plain-text/scan.txt ...,A SCANDAL IN BOHEMIA\n\nArthur Conan ...,A Scandal in Bohemia,51395
/stories/plain-text/redh.txt ...,THE RED-HEADED LEAGUE\n\nArthur Conan ...,The Red-Headed League,54136


It works like pandas Dataframe. We can pass directly a dataframe to the SFrame class.

Let's show the distribution of our text length column

In [10]:
sf['chars_num'].show()

Materializing SArray

Let's calculate the number of words in each text with nltk.

In [11]:
import nltk

Let's use a tokenizer which recuperate only words.

In [12]:
tokenizer = nltk.RegexpTokenizer('\w+')

In [13]:
sf['words_num'] = sf['text'].apply(lambda t: len(tokenizer.tokenize(t)))

In [14]:
sf.head(10)

link,text,title,chars_num,words_num
/stories/plain-text/advs.txt ...,THE ADVENTURES OFSHERLOCK HOLMES\n\n ...,The Adventures ofSherlock Holmes ...,610836,106032
/stories/plain-text/mems.txt ...,THE MEMOIRS OF SHERLOCKHOLMES\n\n ...,The Memoirs of SherlockHolmes ...,511733,88496
/stories/plain-text/retn.txt ...,THE RETURN OF SHERLOCKHOLMES\n\n ...,The Return of SherlockHolmes ...,662236,114457
/stories/plain-text/lstb.txt ...,HIS LAST BOW\n\nArthur Conan ...,His Last Bow,402289,68918
/stories/plain-text/stud.txt ...,A STUDY IN SCARLET\n\nArthur Conan ...,A Study In Scarlet,260045,44205
/stories/plain-text/sign.txt ...,THE SIGN OF THE FOUR\n\nArthur Conan ...,The Sign of the Four,251806,43951
/stories/plain-text/houn.txt ...,THE HOUND OF THEBASKERVILLES\n\n ...,The Hound of theBaskervilles ...,346398,60103
/stories/plain-text/vall.txt ...,THE VALLEY OF FEAR\n\nArthur Conan ...,The Valley of Fear,340222,59098
/stories/plain-text/scan.txt ...,A SCANDAL IN BOHEMIA\n\nArthur Conan ...,A Scandal in Bohemia,51395,8751
/stories/plain-text/redh.txt ...,THE RED-HEADED LEAGUE\n\nArthur Conan ...,The Red-Headed League,54136,9370


In [15]:
# let's use the sent tokenizer to get the sentences but only english chars
def to_sentences(txt: str, remove_none_english_chars: bool = True):
  """Split text into sentences
  """

  for s in nltk.sent_tokenize(txt):
    if remove_none_english_chars:

      # remove non english chars
      s = re.sub("[^a-zA-Z]", " ", s)
    yield s
  

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
# create a new column containing number of sentences of each text
sf["sentences_num"] = sf["text"].apply(lambda t: len(list(to_sentences(t))))

Let's show the statistics of the three columns that we created

In [18]:
sf[["chars_num", "words_num", "sentences_num"]].show()

Materializing SFrame

Let's count the number of times that the words 'Sherlock', 'Watson', and 'Elementary' appeared in each text.

In [19]:
# number of words
sf['words_count'] = tr.text_analytics.count_words(sf['text'], to_lower = True)

sf['sherlowk'] = sf['words_count'].apply(lambda d: d.get('sherlock', 0))

sf['sherlock_count'] = sf['words_count'].apply(lambda d: d.get('sherlock', 0))
sf['watson_count'] = sf['words_count'].apply(lambda d: d.get('watson', 0))
sf['elementary_count'] = sf['words_count'].apply(lambda d: d.get('elementary', 0))


In [20]:
sf.remove_column('sherlowk', inplace=True)

link,text,title,chars_num,words_num,sentences_num
/stories/plain-text/advs.txt ...,THE ADVENTURES OFSHERLOCK HOLMES\n\n ...,The Adventures ofSherlock Holmes ...,610836,106032,6815
/stories/plain-text/mems.txt ...,THE MEMOIRS OF SHERLOCKHOLMES\n\n ...,The Memoirs of SherlockHolmes ...,511733,88496,5502
/stories/plain-text/retn.txt ...,THE RETURN OF SHERLOCKHOLMES\n\n ...,The Return of SherlockHolmes ...,662236,114457,7807
/stories/plain-text/lstb.txt ...,HIS LAST BOW\n\nArthur Conan ...,His Last Bow,402289,68918,5086
/stories/plain-text/stud.txt ...,A STUDY IN SCARLET\n\nArthur Conan ...,A Study In Scarlet,260045,44205,2699
/stories/plain-text/sign.txt ...,THE SIGN OF THE FOUR\n\nArthur Conan ...,The Sign of the Four,251806,43951,2930
/stories/plain-text/houn.txt ...,THE HOUND OF THEBASKERVILLES\n\n ...,The Hound of theBaskervilles ...,346398,60103,3875
/stories/plain-text/vall.txt ...,THE VALLEY OF FEAR\n\nArthur Conan ...,The Valley of Fear,340222,59098,4268
/stories/plain-text/scan.txt ...,A SCANDAL IN BOHEMIA\n\nArthur Conan ...,A Scandal in Bohemia,51395,8751,673
/stories/plain-text/redh.txt ...,THE RED-HEADED LEAGUE\n\nArthur Conan ...,The Red-Headed League,54136,9370,578


Let's describe the three columns.

In [21]:
sf[['sherlock_count', 'watson_count', 'elementary_count']].show()

Materializing SFrame

Let's create a linear regression model which count the number of times that we obtain sherlock word in a text depending on the number of times that we watson word in the text.

In [22]:
sher_wat_reg = tr.linear_regression.create(sf, target = "sherlock_count", features = ['watson_count'])
sher_wat_reg

Linear regression:

--------------------------------------------------------

Number of examples          : 64

Number of features          : 1

Number of unpacked features : 1

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.001022     | 71.035762          | 12.763565                       |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Class                          : LinearRegression

Schema
------
Number of coefficients         : 2
Number of examples             : 64
Number of feature columns      : 1
Number of unpacked features    : 1

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.0019

Settings
--------
Residual sum of squares        : 10426.1504
Training RMSE                  : 12.7636

Highest Positive Coefficients
-----------------------------
(intercept)                    : 3.9686
watson_count                   : 0.2839

Lowest Negative Coefficients
----------------------------
No Negative Coefficients       : 

According to the results we have the following results:

$$
n\_sherlock = n\_watson \times 0.2839 + 3.9686 
$$

##### Constructing Social Networks

1. Constructing Social Network using Names List

Let's recuperate the sentences of each text in a new column sentences.

In [23]:
sf['sentences'] = sf['text'].apply(lambda t: list(to_sentences(t)))

In [24]:
# let's create a s frame which contains for each line a sentence and the title of the sentence
sf_sentences = sf.flat_map(['title', 'text'], lambda t: [[t['title'], s.strip()] for s in to_sentences(t['text'])])

In [25]:
sf_sentences.head()

title,text
The Adventures ofSherlock Holmes ...,THE ADVENTURES OFSHERLOCK HOLMES ...
The Adventures ofSherlock Holmes ...,I have seldom heard himmention her under any ...
The Adventures ofSherlock Holmes ...,In his eyes she eclipsesand predominates ...
The Adventures ofSherlock Holmes ...,It was not that he feltany emotion akin to ...
The Adventures ofSherlock Holmes ...,All emotions and thatone particularly ...
The Adventures ofSherlock Holmes ...,He was I take it themost perfect reasoning ...
The Adventures ofSherlock Holmes ...,He never spoke of thesofter passions ...
The Adventures ofSherlock Holmes ...,They were admirablethings for the ...
The Adventures ofSherlock Holmes ...,But for the trainedreasoner to admit such ...
The Adventures ofSherlock Holmes ...,Grit in a sensitiveinstrument or a crac ...


In [26]:
# rename the second column to sentence
sf_sentences.rename({'text': 'sentence'}, inplace=True)

title,sentence
The Adventures ofSherlock Holmes ...,THE ADVENTURES OFSHERLOCK HOLMES ...
The Adventures ofSherlock Holmes ...,I have seldom heard himmention her under any ...
The Adventures ofSherlock Holmes ...,In his eyes she eclipsesand predominates ...
The Adventures ofSherlock Holmes ...,It was not that he feltany emotion akin to ...
The Adventures ofSherlock Holmes ...,All emotions and thatone particularly ...
The Adventures ofSherlock Holmes ...,He was I take it themost perfect reasoning ...
The Adventures ofSherlock Holmes ...,He never spoke of thesofter passions ...
The Adventures ofSherlock Holmes ...,They were admirablethings for the ...
The Adventures ofSherlock Holmes ...,But for the trainedreasoner to admit such ...
The Adventures ofSherlock Holmes ...,Grit in a sensitiveinstrument or a crac ...


Let's split the sentences into words.

In [27]:
re_words_split = re.compile("(\w+)")
sf_sentences['words'] = sf_sentences['sentence'].apply(lambda s: re_words_split.findall(s))

In [28]:
sf_sentences.head()

title,sentence,words
The Adventures ofSherlock Holmes ...,THE ADVENTURES OFSHERLOCK HOLMES ...,"[THE, ADVENTURES, OF,SHERLOCK, HOLMES, Art ..."
The Adventures ofSherlock Holmes ...,I have seldom heard himmention her under any ...,"[I, have, seldom, heard,him, mention, her, un ..."
The Adventures ofSherlock Holmes ...,In his eyes she eclipsesand predominates ...,"[In, his, eyes, she,eclipses, and, ..."
The Adventures ofSherlock Holmes ...,It was not that he feltany emotion akin to ...,"[It, was, not, that, he,felt, any, emotion, a ..."
The Adventures ofSherlock Holmes ...,All emotions and thatone particularly ...,"[All, emotions, and,that, one, particularly, ..."
The Adventures ofSherlock Holmes ...,He was I take it themost perfect reasoning ...,"[He, was, I, take, it,the, most, perfect, ..."
The Adventures ofSherlock Holmes ...,He never spoke of thesofter passions ...,"[He, never, spoke, of,the, softer, passions, ..."
The Adventures ofSherlock Holmes ...,They were admirablethings for the ...,"[They, were, admirable,things, for, the, ..."
The Adventures ofSherlock Holmes ...,But for the trainedreasoner to admit such ...,"[But, for, the, trained,reasoner, to, admit, ..."
The Adventures ofSherlock Holmes ...,Grit in a sensitiveinstrument or a crac ...,"[Grit, in, a, sensitive,instrument, or, a, cr ..."


In [29]:
# save the results
sf_sentences.save("%s/sentences.sframe" % data_dir)

Let's use the principal name of some characters (the main ones) to recuperate the characters for which their names appear in the same sentences. 

In [30]:
main_characters = set(["Irene", "Mycroft", "Lestrade", "Sherlock", "Moran", "Moriarty", "Watson"])

Let's use the `SGraph`, `Vertex` and `Edge` functions to recuperate the relations between the characters as a graph.

In [31]:
from turicreate import SGraph, Vertex, Edge
import itertools # to iterate through objects and find combinations (order of 2 in our case)
from collections import Counter

In [32]:
def create_graph(s_sentences, characters, min_edge_strength: int = 1):

  s_sentences["characters"] = s_sentences["words"].apply(lambda w: list(set(w) & characters))


  # recuperate line of s_sentences with more than 1 character
  s_sentences['num_characters'] = s_sentences['characters'].apply(lambda ch: len(ch))
  s_sentences = s_sentences[s_sentences['num_characters'] > 1]

  # let's recuperate the links or combinations of characters 
  links = []

  for l in s_sentences['characters']:

    links += itertools.combinations(l, 2)
  
  # recuperate frequencies as a dictionary
  links_freq = Counter(links)

  # We will create an edge between two characters if their frequency is higher than
  # the frequency lower limit
  g = SGraph() # initialize the social network
  edges = [] # initialize the edges

  for link, fr in links_freq.items():

    if fr >= min_edge_strength:

      edges.append(Edge(link[0], link[1], attr = {"strength": fr}))
  
  # return the graph
  return g.add_edges(edges)


In [33]:
# create the graph and render it
g = create_graph(sf_sentences, main_characters)
g

SGraph({'num_edges': 10, 'num_vertices': 7})
Vertex Fields:['__id']
Edge Fields:['__src_id', '__dst_id', 'strength']

In [34]:
g.edges

__src_id,__dst_id,strength
Mycroft,Lestrade,5
Mycroft,Sherlock,6
Lestrade,Mycroft,1
Lestrade,Sherlock,11
Moran,Sherlock,2
Moriarty,Moran,1
Moriarty,Sherlock,4
Moriarty,Watson,6
Irene,Sherlock,4
Watson,Sherlock,14


In [35]:
g.edges[0]

{'__src_id': 'Mycroft', '__dst_id': 'Lestrade', 'strength': 5}

In [36]:
edges = g.edges.apply(lambda row: [v for (k, v) in row.items()])

In [37]:
edges

dtype: list
Rows: 10
[['Mycroft', 'Lestrade', 5], ['Mycroft', 'Sherlock', 6], ['Lestrade', 'Mycroft', 1], ['Lestrade', 'Sherlock', 11], ['Moran', 'Sherlock', 2], ['Moriarty', 'Moran', 1], ['Moriarty', 'Sherlock', 4], ['Moriarty', 'Watson', 6], ['Irene', 'Sherlock', 4], ['Watson', 'Sherlock', 14]]

In [38]:
g.vertices['__id']

dtype: str
Rows: 7
['Mycroft', 'Lestrade', 'Moran', 'Moriarty', 'Irene', 'Sherlock', 'Watson']

Let's create two functions to visualize the graphes. One based on networkx and the other based on `pyviz` which is our preferred. With the last one we will trace a most interactive graph.

In [39]:
!pip install networkx as nx
# it can take few moments to install pyviz
!pip install pyvis==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
# let's import the libraries
# for networkx
import networkx as nx
import matplotlib.pyplot as plt

# for pyviz
from IPython.display import display, HTML
from pyvis.network import Network

from typing import Union


In [110]:
def plot_graph(g, fig_size: tuple = (10, 8), node_size: int = 100, font_size: int = 18, edge_color: str = "blue", node_color: str = "black", alpha: float = .5):
  """ visualize graph with networkx
  """
  edges = g.edges.apply(lambda row: [v for (k, v) in row.items()])
  gnx = nx.Graph()
  gnx.add_nodes_from(g.vertices['__id'])
  gnx.add_weighted_edges_from(edges)

  plt.figure(figsize = fig_size)

  # initialize pos
  pos = nx.kamada_kawai_layout(gnx)

  # Draw the graph
  nx.draw_networkx_edges(gnx, pos, edgelist=edges, edge_color=edge_color, width=[edge[2] for edge in edges], alpha = alpha)
  nx.draw_networkx_nodes(gnx, pos, node_color=node_color, node_size=node_size)
  nx.draw_networkx_labels(gnx, pos, font_size=18, verticalalignment="bottom", horizontalalignment="left")
  plt.show()

def plot_graph2(g, width: str = "500px", height: str = "500px", bgcolor: str = "#ffffff", font_color: bool = False, label_colors: Union[str, list, None] = None, node_distance: int = 90, spring_length: int = 200):
  """Visualize graph with pyviz
  """
  
  # recuperate edges
  edges = g.edges.apply(lambda row: [v for (k, v) in row.items()])
  
  # initialize the network
  net = Network(height, width, bgcolor=bgcolor, font_color=font_color, notebook = True)

  # add the nodes
  nodes = g.vertices['__id'] # recuperate the nodes

  if label_colors: 
    
    label_colors = [label_colors] * len(nodes) if type(label_colors) is str else label_colors

    net.add_nodes(nodes, label = nodes, color = label_colors)

  else:

    net.add_nodes(nodes, label = nodes)

  # add the edges
  net.add_edges(edges)

  # add repulsion options
  net.repulsion(node_distance=node_distance, spring_length=spring_length)

  # add other things in map
  neighbor_map = net.get_adj_list()

  for node in net.nodes:

    node["title"] = f"{node['id']} Neighbors: \n" +"-"+ "\n-".join(neighbor_map[node['id']])

    node["value"] = len(neighbor_map[node["id"]]) 

  # save and show the network
  net.show("net1.html")

  display(HTML("net1.html"))
  




In [111]:
plot_graph2(g)

It seems that Sherlock knows more Watson (her friend) and agent Lestrade. And also he has more relations (with all characters that we mentioned).

Let's make the same thing for the minor characters.

In [112]:
minor_characters = set(["Irene","Mycroft","Lestrade","Sherlock","Moran","Moriarty","Watson","Baynes","Billy","Bradstreet","Gregson"
                            ,"Hopkins","Hudson","Shinwell","Athelney","Mary","Langdale","Toby","Wiggins"])


In [113]:
minor_g = create_graph(sf_sentences, minor_characters)
minor_g

SGraph({'num_edges': 29, 'num_vertices': 15})
Vertex Fields:['__id']
Edge Fields:['__src_id', '__dst_id', 'strength']

In [114]:
plot_graph2(minor_g, "1300px", "800px")

2. Constructing Social Network using Named Entity Recognition

Let's add the `StanfordNERTagger` to recognize automaticaly the entities of our texts. Since it is very complicated to use the stanforNertagger in our case (in colab) let's use the `spacy` library to get the entity names.

In [115]:
# install the spacy model
!python -m spacy download en_core_web_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-18 13:54:51.773653: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [116]:
# from nltk.tag import StanfordNERTagger
import spacy
from spacy.tokens import Doc

# create the natural language processing model
nlp = spacy.load("en_core_web_md")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [117]:
sf_books = tr.load_sframe("%s/books.sframe" % data_dir)

We will use pandarallel to speed up the next transformations.

In [118]:
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
import pandarallel as prl
import pandas as pd

prl.pandarallel.initialize(progress_bar=True, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [120]:
sf_books = sf_books.to_dataframe()

In [121]:
sf_books.head()

,link,text,title
0,/stories/plain-text/advs.txt,THE ADVENTURES OF SHERLOCK HOLMES\n\n ...,The Adventures of Sherlock Holmes
1,/stories/plain-text/mems.txt,THE MEMOIRS OF SHERLOCK HOLMES\n\n ...,The Memoirs of Sherlock Holmes
2,/stories/plain-text/retn.txt,THE RETURN OF SHERLOCK HOLMES\n\n ...,The Return of Sherlock Holmes
3,/stories/plain-text/lstb.txt,HIS LAST BOW\n\n ...,His Last Bow
4,/stories/plain-text/stud.txt,A STUDY IN SCARLET\n\n ...,A Study In Scarlet


Let's change the matcher of the model with a new that we will create to recuperate only the words.

In [122]:
nlp.tokenizer.token_match = re.compile(r"^(\w+)$").match

In [123]:
# insert as parameters the stansford-ner.jar file and the type of classifier we want to use
# st = StanfordNERTagger("/content/drive/MyDrive/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz",
#                        "/content/drive/MyDrive/stanford-ner-2020-11-17/stanford-ner.jar")

# st.java_options = "-Xmx4096m"

In [124]:
sf_books["sentences"] = sf_books["text"].apply(lambda t: list(to_sentences(t)))

In [125]:
# It can take some minutes to get all the entity names
sf_books["NER"] = sf_books["sentences"].parallel_apply(lambda l: [[(t.text, t.label_) for t in d.ents] for d in nlp.pipe(l)])

In [126]:
# ne chunk sents function can create lists of name entities for object of type list(list(str))
# sf_books["NER"] = sf_books["words"].apply(lambda w: [[(token.text, token.ent_type_) for token in Doc(nlp.vocab, sentence)] for sentence in w])
sf_books["person"] = sf_books["NER"].apply(lambda n: [e[0] for s in n for e in s if e[1] == "PERSON"])

person_list = []
for p in sf_books["person"]:
  person_list += p

len(set(person_list))

1810

In [127]:
sf_books.head()

,link,text,title,sentences,NER,person
0,/stories/plain-text/advs.txt,THE ADVENTURES OF SHERLOCK HOLMES\n\n ...,The Adventures of Sherlock Holmes,[THE ADVENTURES OF SHERLOCK HOLMES ...,"[[(Arthur Conan Doyle, PERSON), (The Man with ...","[Arthur Conan Doyle, Irene Adler, Irene Adler,..."
1,/stories/plain-text/mems.txt,THE MEMOIRS OF SHERLOCK HOLMES\n\n ...,The Memoirs of Sherlock Holmes,[THE MEMOIRS OF SHERLOCK HOLMES ...,"[[(Arthur Conan Doyle, PERSON), (Gloria Scott,...","[Arthur Conan Doyle, Gloria Scott, Watson, Hol..."
2,/stories/plain-text/retn.txt,THE RETURN OF SHERLOCK HOLMES\n\n ...,The Return of Sherlock Holmes,[THE RETURN OF SHERLOCK HOLMES ...,"[[(Arthur Conan Doyle, PERSON), (The Adventure...","[Arthur Conan Doyle, Ronald Adair, Sherlock Ho..."
3,/stories/plain-text/lstb.txt,HIS LAST BOW\n\n ...,His Last Bow,[HIS LAST BOW ...,[[(Arthur Conan Doyle ...,"[Sherlock Holmes, John H Watson M, John Scot..."
4,/stories/plain-text/stud.txt,A STUDY IN SCARLET\n\n ...,A Study In Scarlet,[A STUDY IN SCARLET ...,"[[(Arthur Conan Doyle, PERSON), (Sherlock Holm...","[Arthur Conan Doyle, Sherlock Holmes T..."


Let's transform the dataframe that we got to a turi SFrame.

In [128]:
sf_books = tr.SFrame(sf_books)

We can remove some most common names and classified words, etc to make a more readable social network.

In [129]:
c = Counter(person_list) # names' frequencies

In [130]:
characters_set = set(i[0] for i in c.most_common(200)) - set(['the', 'You', 'Mrs', 'He', 'Dr', 'me','did', 'Mr', 
                                      'Now', 'My', 'Miss', 'of', 'Sir', 'Here', 'All', 'Our', 'sir',
                                      'man', 'father', 'What', 'There', 'When', 'no', 'Lord', 'you', 'St',
                                      'John', 'James',  'Holmes', 'Arthur', 'Conan', 'Doyle', 'Lady'])

sf_sentences = tr.load_sframe("%s/sentences.sframe" % data_dir)
g = create_graph(sf_sentences, characters_set)

In [131]:
plot_graph2(g, "1300px", "800px")

Remove the Stanford NER maps 'I' from the graph.

In [132]:
def clean_graph(graph, remove_entities):
  vertices = graph.vertices[graph.vertices["__id"].apply(lambda v: v not in remove_entities)]
  edges = graph.edges[graph.edges.apply(lambda e: e["__src_id"] not in remove_entities and e["__dst_id"] not in remove_entities)]
  return tr.SGraph(vertices, edges)

In [134]:
g = clean_graph(g, {'I'})
plot_graph2(g, "1400px", "900px")

The middle node is Sherlock labeled as `s` due to some loading errors.

Let's recuperate the graph, in this time focusing on the social network's largest component.

In [135]:
def get_graph_largest_component(graph):

  cc = tr.connected_components.create(graph)

  # for each vertice we add her component id
  graph.vertices["component_id"] = cc["graph"].vertices["component_id"]

  # calculate the component_id of the largest component
  largest_component_id = cc["component_size"].sort("Count", ascending = False)[0]['component_id']
  largest_component_vertices = graph.vertices.filter_by(largest_component_id, "component_id")["__id"]
  h = graph.get_neighborhood(largest_component_vertices, 1)
  return h

In [136]:
h = get_graph_largest_component(g)

+-----------------------------+

| Number of components merged |

+-----------------------------+

| 185                         |

| 0                           |

+-----------------------------+

In [137]:
plot_graph2(h, "1500px", "900px")

The above graph is very interesting since we can identify easily relashionships between characters (mains and minors). It seems that all characters excepted some of them have relations with Sherlock Holmes. But the most important relations are between again Watson and Sherlock and Lestrade and Sherlock.

##### Topic Model

We will research for the abstract topics of the different paragraphs of our corpus using something named as `topic model`: According to Wikipedia article, topic model is a type of statistical model for discovering the abstract "topics" that occur in a collection of documents.

In [138]:
# let's separate each story into paragraphs

sf_paragraphs = sf.flat_map(['title', 'text'], lambda t: [[t["title"], p.strip()] for p in t["text"].split("\n\n")])
sf_paragraphs = sf_paragraphs.rename({"text": "paragraph"})

Let's calculate the number of words in each paragraph having less than 25 words.

In [139]:
sf_paragraphs["paragraph_words_number"] = sf_paragraphs["paragraph"].apply(lambda p: len(tokenizer.tokenize(p)))
sf_paragraphs = sf_paragraphs[sf_paragraphs["paragraph_words_number"] >= 25]

Let's use the `turicreate`'s topic model toolkit to discover the topic of each paragraph (10 topics exactly). The topics can change at each trial.

In [140]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [141]:
docs = tr.text_analytics.count_ngrams(sf_paragraphs["paragraph"], n = 1)
stopwords = set(nltk.corpus.stopwords.words())

# let's add some other stopwords 
stopwords |= set(['man', 'mr', 'sir', 'make', 'made', 'll', 'door', 'long', 'day', 'small'])
docs = docs.dict_trim_by_keys(stopwords, exclude = True)
docs = docs.dropna()
topic_model = tr.topic_model.create(docs, num_topics=10)

Learning a topic model

Number of documents     11657

Vocabulary size     18007

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 953.355ms     | 3.61358e+06    | 0               |

+-----------+---------------+----------------+-----------------+

Let's view the most common words in each topic.

In [142]:
topic_model.get_topics().print_rows(1000)

# save the model
topic_model.save("%s/topic_model" % data_dir)

+-------+----------+-----------------------+
| topic |   word   |         score         |
+-------+----------+-----------------------+
|   0   |   room   |  0.02768494826334979  |
|   0   |  watson  |  0.017243524951659924 |
|   0   |   eyes   |  0.014299286401561327 |
|   0   |   mrs    |  0.011263988927232878 |
|   0   |  death   |  0.009655281265838799 |
|   1   |  years   |  0.012905635450522087 |
|   1   |  london  |  0.011825278500586012 |
|   1   |  night   |  0.008028595505096376 |
|   1   | business |  0.007843391456535904 |
|   1   | country  | 0.0074729833594149656 |
|   2   |   back   |  0.030831329708564933 |
|   2   |  father  |  0.011196328129639086 |
|   2   |  watson  |  0.010383414051530692 |
|   2   |  words   |  0.009132777008287006 |
|   2   |  facts   |  0.008194799225854244 |
|   3   |  holmes  |  0.08111172729158318  |
|   3   |  black   |  0.013417932504482135 |
|   3   |   knew   |  0.013139102228471193 |
|   3   |   head   |  0.012550460534670314 |
|   3   | 

Let's visualize more clearly the topics and their most common words with the excellent `pyLDAvis` package developed by Ben Mabey. We will then use the `sklearn` topic model named as `LatentDirichletAllocation` because the `pyLDAvis.graphlab` module is not anymore available.

In [143]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Let's create our new topic model with 10 components like we did previously.

In [144]:
# Let's import the sklearn tools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # for vectorization
from sklearn.decomposition import LatentDirichletAllocation # for training the topic model

Let's get the documents which are the paragraphs.

In [145]:
documents = sf_paragraphs['paragraph']

Let's vectorize the corpus.

In [146]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stopwords,
                                lowercase = True,
                                token_pattern = r'\b(\w+)\b',
                                )

dtm_tf = tf_vectorizer.fit_transform(documents)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(documents) # recuperate results as a one dimensionnal array

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['acesti', 'aci', 'aiba', 'aixi', 'aixo', 'aldiz', 'alli', 'altal', 'altalaban', 'altmıs', 'ambdos', 'amig', 'aqui', 'arrol', 'astia', 'ate', 'aveti', 'avra', 'avro', 'azert', 'aztan', 'azutan', 'bade', 'bae', 'baiknya', 'baizik', 'bar', 'belul', 'berkali', 'bes', 'birkac', 'birsey', 'bos', 'brzcas', 'brzkone', 'bukatzeko', 'butun', 'caci', 'cadascu', 'carei', 'caror', 'carui', 'cate', 'cati', 'catva', 'cem', 'cemer', 'cemerkoli', 'cemu', 'cemur', 'cemurkoli', 'ceprav', 'ceravno', 'cesa', 'cesar', 'cesarkoli', 'cetrta', 'cetrte', 'cetrtega', 'cetrtem', 'cetrtemu', 'cetrti', 'cetrtih', 'cetrtim', 'cetrtima', 'cetrtimi', 'cetrto', 'cetudi', 'cez', 'cezenj', 'ceznje', 'cigar', 'cigav', 'cigava', 'cigave', 'cigavega', 'cigavem', 'cigavemu', 'cigavi', 'cigavih', 'cigavim', 'cigavima', 'cigavimi', '

In [147]:
print(f"tfidf vector: {dtm_tfidf.data}") # print the vector for the tfidf
print(f"tf vector: {dtm_tf.data}") # print the vector for the tf

tfidf vector: [0.12573468 0.12874305 0.12573468 ... 0.27078832 0.27078832 0.26511483]
tf vector: [1 1 1 ... 1 1 1]


Let's train the topic model.

In [148]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(random_state=0)

Let's visualize the results with `pyLDAvis`.

In [150]:
import pyLDAvis
# import pyLDAvis.graphlab # graphlab is not longer available we will use sklearn so
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.088699  0.022684       1        1  18.026653
3     -0.015722  0.011447       2        1  13.167647
0     -0.031317  0.029850       3        1   9.691394
9      0.200060  0.032016       4        1   9.373633
1     -0.091282  0.034242       5        1   8.922901
5     -0.023492 -0.017314       6        1   8.679948
8     -0.000528 -0.071894       7        1   8.561286
7     -0.021004  0.027289       8        1   8.467656
6      0.024472 -0.139831       9        1   7.886317
2      0.047512  0.071513      10        1   7.222564, topic_info=             Term         Freq        Total Category  logprob  loglift
16017        text   255.000000   255.000000  Default  30.0000  30.0000
17473      watson  1312.000000  1312.000000  Default  29.0000  29.0000
15369      street   599.000000   599.000000  Default  28.0000  28.0000
17435  warranties   124.000000   124.000000  Default  27.0000  27.0000
13560        room  1430.000000  1430.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
9089         lady    51.761712   550.083083  Topic10  -6.0465   0.2645
2526         case    57.181285   964.095576  Topic10  -5.9469  -0.1970
7468         hand    55.908681   976.731664  Topic10  -5.9694  -0.2325
5970         eyes    53.421992   816.300563  Topic10  -6.0149  -0.0986
6629        found    46.352468   906.421272  Topic10  -6.1569  -0.3453

[887 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
23         2  0.118844     15
23         9  0.831906     15
74         9  0.891661   1915
84         7  0.951199   221b
149        3  0.914561     85
...      ...       ...    ...
17961      7  0.053576  young
17961      8  0.042546  young
17961      9  0.115031  young
17961     10  0.130788  young
17973      7  0.855425  zamba

[2779 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 10, 2, 6, 9, 8, 7, 3])

From the above visualization we can make the conclusion that most of the topics refear to `Holmes` because the word Holmes appeared on top of the most relevant terms of each topic excepted for the topic number `4` which have, as the most relevant term, the word `case` and the topic number `9` which refears mostly to `Watson` than Holmes.

##### Finding Similar Paragraphs using Word2Vec

Let's include some `deep learning` staffs 👌!. The purpose is to search for paragraphs that have similar text or writing style.

Let's use the `Word2Vec` model using the `gensim` package. The gensim model seems to not work correctly in this environment so let's use `spacy`.

In [151]:
# import gensim

# use sherlock's stories
txt = urllib.request.urlopen("https://sherlock-holm.es/stories/plain-text/cnus.txt").read()

# decode the text to utf8
txt = txt.decode('utf8')



In [152]:
# create the natural language processing model
nlp = spacy.load("en_core_web_md", exclude=['ner'])

In [160]:
nlp.max_length

1000000

In [161]:
nlp.tokenizer.token_match = re.compile(r"^(\w+)$").match # add a new regex matcher

In [162]:
def get_tokenized_sentences(txt: str):

  # use spacy to recuperate the tokens
  sentences = nltk.sent_tokenize(txt)

  for sent in nlp.pipe(sentences):

    yield [t.text for t in sent]

In [163]:
# sents = list(get_tokenized_sentences(txt))

We want to get similarities between some words of the text. Let's transform the text to a spacy doc object. We want to use only a part of the text because it is very large. Say lower than 1000000 first letters (max length of nlp).

In [164]:
len(txt)

3381928

In [168]:
doc = nlp(txt[:500000].lower()) # convert to lowercase because creating the doc object

Let's get the 30 most similar words of "watson" and "holmes".

In [169]:
word1 = "watson"
word2 = "holmes"

In [177]:
import numpy as np

In [191]:

# let's create a function for getting the most similar words of a given word
def get_most_similar(doc, word: str, n: int = 30):

  # vectorize vocab to recuperate most similar words with distance matching
  ms = doc.vocab.vectors.most_similar(
      np.asarray([doc.vocab.vectors[doc.vocab.strings[word]]]), n = n
      )
  # recuperate words
  words = [doc.vocab.strings[w] for w in ms[0][0]]
  
  # return results as DataFrame
  return pd.DataFrame({"words": words, "distances": ms[2][0]})

In [192]:
get_most_similar(doc, word1)

,words,distances
0,Mimblewimble,1.0000
1,Renson,0.8140
2,Latshaw,0.8068
3,Armathwaite,0.8068
4,Poneman,0.8062
5,Kerhonkson,0.7938
6,Wallidan,0.7914
7,Davik,0.7902
8,LZ4,0.7874
9,Macker,0.7843


In [193]:
get_most_similar(doc, word2)

,words,distances
0,Woos,1.0000
1,Mimblewimble,0.7320
2,Loyle,0.6660
3,TOPSIS,0.6625
4,Armathwaite,0.6319
5,Wallidan,0.6276
6,Macker,0.6275
7,Modot,0.6268
8,LZ4,0.6246
9,Spraggett,0.6235


Let's recuperate the mean vector of the paragraphs.

In [194]:
# we will use the sf_paragraph frame
def get_mean_vector(paragraph: str):
  doc_p = nlp(paragraph)
  return np.mean(doc_p.vector)

# let's get the mean of paragraphs using pandas_parallel
sf_paragraphs = sf_paragraphs.to_dataframe() # convert to a DataFrame

sf_paragraphs['mean_vector'] = sf_paragraphs['paragraph'].parallel_apply(get_mean_vector)

Retransform the frame and train the nearest neighbors model.

In [ ]:
sf_paragraphs = tr.SFrame(sf_paragraphs)
nn_model = tr.nearest_neighbors.create(sf_paragraphs, features = ["mean_vector"])

# getting the two nearest neighbors of each paragraph from all the paragraphs
r = nn_model.query(sf_paragraphs, k = 2)
r.head(10)

Starting ball tree nearest neighbors model training.

+------------+--------------+

| Tree level | Elapsed Time |

+------------+--------------+

| 0          | 6.55ms       |

...

+--------------+-------------+--------------+

| 1            | 0           | 1.784ms      |

| Done         |             | 886.954ms    |

+--------------+-------------+--------------+

query_label,reference_label,distance,rank
0,0,0.0,1
0,6231,0.0008178725838661194,2
1,1,0.0,1
1,6453,0.0,2
2,2,0.0,1
2,6454,0.0,2
3,6455,0.0,1
3,3,0.0,2
4,4,0.0,1
4,6456,0.0,2


Let's recuperate paragraphs having distances different from 0 (not the same paragraphs) and which are too distant.

In [ ]:
r = r[(r['distance'] < 0.08) & (r['distance'] != 0)]
r.head()

query_label,reference_label,distance,rank
0,6231,0.0008178725838661194,2
1166,5694,8.863612310960889e-05,2
2149,4359,7.35744833946228e-05,2
2150,8921,0.00058756023645401,2
3326,7925,0.001023031771183014,2
3327,2791,2.1470244973897934e-05,2
4083,10644,1.4334917068481445e-05,2
4084,11156,2.1391548216342926e-05,2
4085,6388,2.818182110786438e-05,2
4086,4140,1.1101365089416504e-05,2


Let's join the paragraphs on columns "query_label" and "reference_label".

In [ ]:
sf_paragraphs = sf_paragraphs.add_row_number("query_label")
sf_paragraphs = sf_paragraphs.add_row_number("reference_label")
sf_similar = r.join(sf_paragraphs, on = "query_label").join(sf_paragraphs, on = "reference_label")

In [ ]:
sf_similar[["paragraph", "title", "title.1", "paragraph.1", "distance"]]

paragraph,title,title.1,paragraph.1
Miller Hill is an ill-kept public park in the ...,The Valley of Fear,The Adventures ofSherlock Holmes ...,A Scandal in Bohemia\nThe Red-Headed League\n ...
"""All this seems strangeto you,"" continued ...",A Study In Scarlet,The Adventures ofSherlock Holmes ...,To Sherlock Holmes she isalways the woman. I have ...
He took up a secondletter which had lain ...,The Sussex Vampire,The Adventures ofSherlock Holmes ...,I had seen little ofHolmes lately. My ...
"As he spoke the bright,yellow light in the s ...",The Valley of Fear,The Adventures ofSherlock Holmes ...,His manner was noteffusive. It seldom was; ...
"""But why in the worldwould anyone want to ...",Shoscombe Old Place,The Adventures ofSherlock Holmes ...,His manner was noteffusive. It seldom was; ...
"""You've been so kind tous, sir, that I should ...",The Hound of theBaskervilles ...,The Adventures ofSherlock Holmes ...,"""Indeed, I should havethought a little more. ..."
It appeared to have beenfitted up as a chemical ...,The Sign of the Four,The Adventures ofSherlock Holmes ...,"I did so, and saw a large""E"" with a small ""g,"" a ..."
"Well, I happen to beclear to-morrow ...",The Three Garridebs,The Adventures ofSherlock Holmes ...,"""Never mind him. I maywant your help, and so ..."
"""You compliment me,madam. At the same time, ...",The Veiled Lodger,The Adventures ofSherlock Holmes ...,"""Never mind him. I maywant your help, and so ..."
"""The whole course ofevents,"" said Holmes, ...",The Hound of theBaskervilles ...,The Adventures ofSherlock Holmes ...,A man entered who couldhardly have been less ...


Let's look at some the similar paragraphs.

In [ ]:
print(sf_similar[1]["paragraph"])
print("-"*100)
print(sf_similar[1]["paragraph.1"])

"All this seems strange to you," continued Holmes, "because you
     failed at the beginning of the inquiry to grasp the importance of the
     single real clue which was presented to you. I had the good fortune
     to seize upon that, and everything which has occurred since then has
     served to confirm my original supposition, and, indeed, was the
     logical sequence of it. Hence things which have perplexed you and
     made the case more obscure, have served to enlighten me and to
     strengthen my conclusions. It is a mistake to confound strangeness
     with mystery. The most commonplace crime is often the most mysterious
     because it presents no new or special features from which deductions
     may be drawn. This murder would have been infinitely more difficult
     to unravel had the body of the victim been simply found lying in the
     roadway without any of those outré and sensational accompaniments
     which have rendered it remarkable. These strange details, far 

# END!